# Word Generator

This program will use Mantas Lukoševičius' ESN to try to generate new words, from an input text. While the main program is explained in the "Minimal ESN - EN" notebook, we will here focus on the added parts that will help achieving this task.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from ipywidgets import *
from IPython.display import *

def set_seed(seed=None):
    """Making the seed (for random values) variable if None"""

    if seed is None:
        import time
        seed = int((time.time()*10**6) % 4294967295)
        print(seed)
    try:
        np.random.seed(seed)
        print("Seed used for random values:", seed)
    except:
        print("!!! WARNING !!!: Seed was not set correctly.")
    return seed

class Network(object):

    def __init__(self, trainLen=2000, testLen=2000, initLen=100) :
        self.initLen = initLen
        self.trainLen = trainLen
        self.testLen = testLen
        self.file = open("text/Shakespeare.txt", "r").read()
        self.resSize = 300
        self.a = 0.3
        self.spectral_radius = 1.25
        self.input_scaling = 1.
        self.reg =  1e-8
        self.mode = 'prediction'
        seed = None #42
        set_seed(seed)
        
nw = Network()

2237381921
Seed used for random values: 2237381921


The next function analyzes a text (e.g., Sir Arthur Conan Doyle's Sherlock Holmes), and returns a list containing all the different characters that are present in the text. You can choose between taking case, punctuation and/or numbers into account.

In [2]:
def filter_characters(nw, keep_upper=True, keep_punctuation=True, keep_numbers=True) :

    alphabet = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 ")
    numbers = list("0123456789")

    if keep_upper == False : nw.file = nw.file.lower()
    nw.input_text = list(nw.file)

    if keep_punctuation == False :
        nw.input_text = [i for i in nw.input_text if i in alphabet]     

    if keep_numbers == False :
        nw.input_text = [i for i in nw.input_text if i not in numbers]
    
    return(nw)

Let's see what characters are in the chosen input text.

In [3]:
def characters(nw) :
    nw.input_units, nw.output_units = dict(), dict()
    for i, item in enumerate(set(nw.input_text)) : nw.input_units[item] = i
    for i, item in enumerate(set(nw.input_text)) : nw.output_units[i] = item
    #nw.input_units = dict(enumerate(set(nw.input_text)))
    print("\nExisting characters in the text :", sorted(nw.input_units),
          "\nNumber of different characters :", len(nw.input_units), "\n")
    return(nw)

We will then convert the text values into numerical values, usable by the algorithm. We will consider the input as a vector $u(t)$, where each line matches a different character, according to <b>nw.chars</b>. Since there only can be one character at a time, we will convert the text from nw.input_text into a nw.data vector, where each element will be a character ID according to its position in nw.chars.

In [4]:
def convert_input(nw) :
    print("Converting input into ID numbers...", end=" ")
    nw.data = np.array([nw.input_units[i] for i in nw.input_text])
    nw.inSize = nw.outSize = len(nw.input_units)
    print("done.")
    return(nw)

Now, we can try this on our network. The input $u$ will now be a vector, matching the size of nw.chars. Every time the program "reads" a character, we will give the corresponding neuron a value of 1. In any other case, this value will be 0.

In [5]:
def binary_data(nw) :
    print("Creating the input binary matrix...", end=" ")
    nw.data_b = np.zeros((len(nw.input_text), len(nw.input_units)))
    for i, item in enumerate(nw.data) :
        nw.data_b[i][item] = 1
    print("done.\n") 
    return(nw)

Then, we copy the code from the ESN.

In [6]:
def initialization(nw) :
    print("\nInitializing the network matrices...", end=" ")
    set_seed()
    nw.Win = (np.random.rand(nw.resSize,1+nw.inSize)-0.5) * nw.input_scaling
    nw.W = np.random.rand(nw.resSize,nw.resSize)-0.5 
    nw.X = np.zeros((1+nw.inSize+nw.resSize,nw.trainLen-nw.initLen))
    nw.Ytarget = nw.data_b[nw.initLen+1:nw.trainLen+1].T
    nw.x = np.zeros((nw.resSize,1))
    print("done.")
    return(nw)

def compute_spectral_radius(nw):
    print('Computing spectral radius...',end=" ")
    rhoW = max(abs(linalg.eig(nw.W)[0]))
    print('done.')
    nw.W *= nw.spectral_radius / rhoW
    return(nw)

def train_network(nw) :
    print('Training the network...', end=" ")
    percent = 0.1
    for t in range(nw.trainLen):
        percent = progression(percent, t, nw.trainLen)
        nw.u = nw.data_b[t%len(nw.data)]
        nw.x = (1-nw.a)*nw.x + nw.a*np.tanh( np.dot(nw.Win, np.concatenate((np.array([1]),nw.u)).reshape(len(nw.input_units)+1,1) ) + np.dot( nw.W, nw.x ) )
        if t >= nw.initLen :
            nw.X[:,t-nw.initLen] = np.concatenate((np.array([1]),nw.u,nw.x[:,0])).reshape(len(nw.input_units)+nw.resSize+1,1)[:,0]      
    print('done.')
    return(nw)

def train_output(nw) :
    print('Training the output...', end=" ")
    nw.X_T = nw.X.T
    if nw.reg is not None:
        nw.Wout = np.dot(np.dot(nw.Ytarget,nw.X_T), linalg.inv(np.dot(nw.X,nw.X_T) + \
            nw.reg*np.eye(1+nw.inSize+nw.resSize) ) )
    else:
        nw.Wout = np.dot(nw.Ytarget, linalg.pinv(nw.X) )   
    print('done.')
    return(nw)

def test(nw) :
    print('Testing the network... (', nw.mode, ' mode)', sep="", end=" ")
    nw.Y = np.zeros((nw.outSize,nw.testLen))
    nw.u = nw.data_b[nw.trainLen%len(nw.data)]
    percent = 0.1
    for t in range(nw.testLen):
        percent = progression(percent, t, nw.trainLen)
        nw.x = (1-nw.a)*nw.x + nw.a*np.tanh( np.dot(nw.Win, np.concatenate((np.array([1]),nw.u)).reshape(len(nw.input_units)+1,1)\
                                                   ) + np.dot(nw.W,nw.x ) )
        nw.y = np.dot(nw.Wout, np.concatenate((np.array([1]),nw.u,nw.x[:,0])).reshape(len(nw.input_units)+nw.resSize+1,1)[:,0] )
        nw.Y[:,t] = nw.y
        if nw.mode == 'generative':
            # generative mode:
            nw.u = nw.y
        elif nw.mode == 'prediction':
            ## predictive mode:
            nw.u = np.zeros(len(nw.input_units))
            nw.u[nw.data[(nw.trainLen+t+1)%len(nw.data)]] = 1
        else:
            raise(Exception, "ERROR: 'mode' was not set correctly.")
    print('done.\n')
    return(nw)

def compute_error(nw) :
    print("Computing the error...", end=" ")
    errorLen = 500
    mse = sum( np.square( nw.data[(nw.trainLen+1)%len(nw.data):(nw.trainLen+errorLen+1)%len(nw.data)] - nw.Y[0,0:errorLen] ) ) / errorLen
    print('MSE = ' + str( mse ))
    return(nw)

When the output is ready, it's only characters IDs. The next function will "translate" those ID into the corresponding characters.

In [7]:
def convert_output(nw) :
    nw.output_text = ""
    print("Converting the output...", end=" ")
    for i in range(len(nw.Y.T)) :
        nw.output_text += nw.output_units[nw.Y.T[i].argmax()]
    print("done.")
    return(nw.output_text)
    
def progression(percent, i, total) :
    if i == 0 :
        print("Progress :", end= " ")
        percent = 0.1
    elif (i/total) > percent :
        print(round(percent*100), end="")
        print("%", end=" ")
        percent += 0.1
    if total-i == 1 :
        print("100%")
    return(percent)

def compute_network(nw, a, b, c) :
    nw = filter_characters(nw, a, b, c)
    nw = characters(nw)
    nw = convert_input(nw)
    nw = binary_data(nw)
    nw = initialization(nw)
    nw = compute_spectral_radius(nw)
    nw = train_network(nw)
    nw = train_output(nw)
    nw = test(nw) 
    nw = compute_error(nw)
    nw = convert_output(nw)
    return(nw)

## Configure network

Let's configure the network's variables you want to use.

In [12]:
select_mode = ToggleButtons(description='Mode:', options=['prediction', 'generative'])
text = Dropdown(description='Text:', options={"Shakespeare (4 573 338 chars.)" : 1, "Sherlock Holmes (3 868 223 chars)" : 2,
                                                   "Harry Potter and the Sorcerer's Stone (439 743 chars.)" : 3,
                                                   "Harry Potter and the Prisoner of Azkaban (611 584 chars.)" : 4}, value = 4)
var1 = Checkbox(description='Keep upper case letters?',value=True,)
var2 = Checkbox(description='Keep punctuation?',value=True,)
var3 = Checkbox(description='Keep numbers?',value=True,)
var4 = FloatSlider(value=300, min=0, max=5000, step=1, description='resSize')
var5 = FloatText(value=1000, description='initLen')
var6 = FloatText(value=10000, description='trainLen')
var7 = FloatText(value=2000, description='testLen')
var8 = FloatSlider(value=1.25, min=0, max=10, step=0.05, description='spectral radius')
var9 = FloatSlider(value=0.3, min=0, max=1, step=0.01, description='leak rate')
valid = Button(description='Validate')

def record_values(_):
    clear_output()
    nw.mode=select_mode.value
    file = text.value
    texts = ["Shakespeare.txt", "SherlockHolmes.txt", "HarryPotter1.txt", "HarryPotter3.txt"]
    nw.file = open("text/"+texts[file-1], "r").read()
    nw.resSize=int(var4.value)
    nw.initLen=int(var5.value)
    nw.trainLen=int(var6.value)
    nw.testLen=int(var7.value)
    nw.spectral_radius=float(var8.value)
    nw.a=float(var9.value)
    
    print("InitLen:", nw.initLen, "TrainLen:", nw.trainLen, "TestLen:", nw.testLen) 
    print("ResSize:", nw.resSize, "Spectral Radius:", nw.spectral_radius, "Leak Rate:", nw.a)
    
    compute_network(nw, var1.value, var2.value, var3.value)
    return(nw.output_text)

display(select_mode)
display(text)
display(var1)
display(var2)
display(var3)
display(var4)
display(var5)
display(var6)
display(var7)
display(var8)
display(var9)
display(valid)

output_text = valid.on_click(record_values)

InitLen: 1000 TrainLen: 1000000 TestLen: 10000
ResSize: 2057 Spectral Radius: 1.25 Leak Rate: 0.75

Existing characters in the text : ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] 
Number of different characters : 67 

Converting input into ID numbers... done.
Creating the input binary matrix... done.


Initializing the network matrices... 2318020224
Seed used for random values: 2318020224
done.
Computing spectral radius... 

The next graph shows the characters repartition throughout the first characters of the text. The x-axis represents the first n characters of the text (which you can change), and the y-axis represents every different character in the text (generally ~70, counting upper case, lower case, numbers and punctuation symbols)..

In [13]:
var10 = FloatText(value=1000, description='Number of chars')
valid2 = Button(description='Validate')
        
def trace_graph1(_) :
    clear_output()
    plt.figure(1).clear()
    limit = np.arange(int(var10.value))
    plt.plot(limit,nw.data[0:int(var10.value)], 'ro')
    plt.title('Characters throughout the text')
    
valid2.on_click(trace_graph1)
    
display(var10)
display(valid2)

Finally, let's see the ouput text the program generates !

In [14]:
valid3 = Button(description='Show the output!')

def show_output(_) :
    print(nw.output_text)

display(valid3)
valid3.on_click(show_output)

AttributeError: 'Network' object has no attribute 'output_text'